# Anova Analysis of ~55k genes dataset


Import all requisite libraries. 

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
import patsy

from sklearn import manifold
from sklearn.manifold import TSNE
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.ticker import NullFormatter
from sklearn.decomposition import PCA

from time import time

Load the data

In [2]:
#load the data 
df = pd.read_csv('genename_counts_nooutliers.tsv', sep='\t')
dfn = pd.DataFrame()
##dfn['age'] = df['age']
##dfn['mgs_level'] = df['mgs_level']
df.rename(columns={df.columns[0]: "gene" }, inplace = True)
df = df.set_index('gene')

## See whether we have the table in the right format
df.head()

,100_2,101_3,102_2,103_3,104_2,105_2,106_4,107_4,109_1,11_4,...,90_2,91_2,92_3,93_2,94_4,95_4,96_3,97_2,98_3,99_1
gene,,,,,,,,,,,,,,,,,,,,,
TSPAN6,225.00,252.0,136.0,166.0,207.0,121.0,127.0,304.0,227.0,224.0,...,132.0,149.0,186.0,71.0,272.0,136.0,324.0,158.0,168.0,167.0
TNMD,0.00,0.0,1.0,4.0,0.0,0.0,0.0,0.0,4.0,0.0,...,0.0,0.0,3.0,1.0,1.0,1.0,1.0,2.0,1.0,0.0
DPM1,254.00,301.0,173.0,264.0,307.0,140.0,164.0,279.0,216.0,274.0,...,369.0,148.0,265.0,86.0,326.0,283.0,300.0,242.0,286.0,207.0
SCYL3,422.99,510.0,272.0,301.0,417.0,116.0,198.0,278.0,243.0,297.0,...,274.0,163.0,227.0,96.0,418.0,338.0,277.0,342.0,343.0,140.0
C1orf112,272.00,310.0,204.0,224.0,227.0,149.0,253.0,171.0,274.0,198.0,...,280.0,168.0,308.0,86.0,284.0,260.0,179.0,210.0,225.0,124.0


In [16]:
# Make sure to drop NA
dfn = df.dropna()

## Get the max library depth
max_ld = dfn.sum(axis=0).max()

## Multiply by max depth
new_df = dfn.multiply(max_ld)

## sum across the column, then divide by each library depth
dfn = new_df.div(dfn.sum(axis=0), axis=1)

dfn.head()
    

,100_2,101_3,102_2,103_3,104_2,105_2,106_4,107_4,109_1,11_4,...,90_2,91_2,92_3,93_2,94_4,95_4,96_3,97_2,98_3,99_1
gene,,,,,,,,,,,,,,,,,,,,,
TSPAN6,493.964287,578.967391,515.801226,488.194268,515.851176,374.777410,490.193679,964.448360,736.753958,800.584324,...,402.528799,632.336474,621.280134,522.965367,811.551362,492.816705,1102.405799,440.183351,496.517354,580.809933
TNMD,0.000000,0.000000,3.792656,11.763717,0.000000,0.000000,0.000000,0.000000,12.982449,0.000000,...,0.000000,0.000000,10.020647,7.365709,2.983645,3.623652,3.402487,5.571941,2.955460,0.000000
DPM1,557.630795,691.544383,656.129501,776.405342,765.054642,433.626755,633.006010,885.135173,701.052225,979.286181,...,1125.250961,628.092605,885.157181,633.451007,972.668176,1025.493584,1020.746110,674.204880,845.261685,719.926084
SCYL3,928.630906,1171.719719,1031.602452,885.219727,1039.178455,359.290740,764.238963,881.962645,788.683753,1061.489036,...,835.552204,691.750639,758.228981,707.108101,1247.163490,1224.794457,942.488908,952.801937,1013.722930,486.906531
C1orf112,597.147938,712.221790,773.701839,658.768169,565.691869,461.502761,976.527564,542.502203,889.297730,707.659357,...,853.848968,712.969984,1028.786459,633.451007,847.355098,942.149582,609.045179,585.053821,664.978599,431.260070


### When row is standardized using mean and sigma, we get all NaN for p values. It could be that that messes up the variance, but I'm not exactly sure why.

In [17]:
## This gave me NaN's for all p-values
# ## Mean across row
ave = dfn.mean(axis=1)

# ## Sigma across row
sig = dfn.std(axis=1)

# ## Subtract the mean from the row
df_sub = dfn.sub(dfn.mean(axis=1), axis=0)

# ## Divide by sigma
dfn = df_sub.div(df_sub.std(axis=1), axis=0)

In [22]:
dfn.head()

,100_2,101_3,102_2,103_3,104_2,105_2,106_4,107_4,109_1,11_4,...,90_2,91_2,92_3,93_2,94_4,95_4,96_3,97_2,98_3,99_1
gene,,,,,,,,,,,,,,,,,,,,,
TSPAN6,-0.671384,-0.213996,-0.553883,-0.702432,-0.553615,-1.312710,-0.691673,1.860214,0.635028,0.978488,...,-1.163384,0.073174,0.013682,-0.515334,1.037500,-0.677559,2.602541,-0.960771,-0.657647,-0.204082
TNMD,-0.790724,-0.790724,0.298657,2.588220,-0.790724,-0.790724,-0.790724,-0.790724,2.938281,-0.790724,...,-0.790724,-0.790724,2.087550,1.324961,0.066281,0.250113,0.186587,0.809729,0.058186,-0.790724
DPM1,-1.141165,-0.326096,-0.541650,0.190413,0.121327,-1.895920,-0.682391,0.852201,-0.268226,1.425254,...,2.313674,-0.712297,0.852335,-0.679683,1.384973,1.706497,1.677601,-0.431633,0.609510,-0.153350
SCYL3,-0.312309,0.659636,0.099403,-0.485881,0.129695,-2.588710,-0.969600,-0.498903,-0.871862,0.218899,...,-0.684467,-1.259431,-0.993630,-1.198027,0.961284,0.871846,-0.256900,-0.215666,0.027916,-2.078462
C1orf112,-0.668913,-0.129821,0.158198,-0.380238,-0.816277,-1.304377,1.108386,-0.924915,0.699736,-0.151195,...,0.533667,-0.126316,1.353206,-0.498842,0.503245,0.947334,-0.613178,-0.725571,-0.351143,-1.446056


In [18]:
## Split the data into different groups for ANOVA testing
train_stage1 = {}
train_stage2 = {}
train_stage3 = {}
train_stage4 = {}

for col in dfn.columns:
    if '_1' in col:
        train_stage1[col] = dfn[col]
    elif '_2' in col:
        train_stage2[col] = dfn[col]
    elif '_3' in col:
        train_stage3[col] = dfn[col]
    elif '_4' in col:
        train_stage4[col] = dfn[col]


# train_stage1['gene'] = dfn['gene']
# train_stage2['gene'] = dfn['gene']
# train_stage3['gene'] = dfn['gene']
# train_stage4['gene'] = dfn['gene']

train_stage1 = pd.DataFrame(train_stage1).transpose()
train_stage2 = pd.DataFrame(train_stage2).transpose()
train_stage3 = pd.DataFrame(train_stage3).transpose()
train_stage4 = pd.DataFrame(train_stage4).transpose()

In [19]:
train_stage1.head()

gene,TSPAN6,TNMD,DPM1,SCYL3,C1orf112,FGR,CFH,FUCA2,GCLC,NFYA,...,MIR3116-2,Y_RNA,RP11-158M9.1,IGHVII-20-3,MIR3202-2,CTD-2331H12.9,RP11-122G18.12,RP5-937E21.8,RP11-606M12.1,MIR4481
109_1,0.635028,2.938281,-0.268226,-0.871862,0.699736,0.052754,0.117716,0.398348,-0.667476,0.929808,...,NaN,-0.046984,-1.024798,-0.066059,NaN,NaN,-1.382021,-0.142701,-0.432482,NaN
115_1,-0.319279,-0.170341,0.473422,0.154440,-0.734446,-0.806619,-1.110267,-0.380956,-0.541218,0.600746,...,NaN,-0.046984,-0.564145,-0.066059,NaN,NaN,-0.264141,-0.142701,-0.432482,NaN
117_1,-0.373346,-0.790724,0.912326,0.182608,1.015679,0.491173,1.503529,-0.055941,-0.149443,0.551055,...,NaN,-0.046984,0.291675,-0.066059,NaN,NaN,-1.382021,-0.142701,-0.432482,NaN
119_1,0.158826,1.566250,0.609088,-1.433343,2.588197,0.413581,0.794765,0.669020,0.167047,3.138881,...,NaN,-0.046984,-0.149736,-0.066059,NaN,NaN,-1.382021,-0.142701,5.637019,NaN
120_1,-0.308599,0.361057,0.785390,-1.508111,-0.517097,0.264206,0.385319,-0.543899,-0.119337,-0.255366,...,NaN,-0.046984,1.540898,-0.066059,NaN,NaN,-1.382021,-0.142701,-0.432482,NaN


In [20]:
train_stage1 = train_stage1.dropna(axis = 1 , how = 'all')
train_stage2 = train_stage2.dropna(axis = 1 , how = 'all')
train_stage3 = train_stage3.dropna(axis = 1 , how = 'all')
train_stage4 = train_stage4.dropna(axis = 1 , how = 'all')

In [24]:
test = pd.concat( [train_stage1, train_stage2, train_stage3, train_stage4], axis=0)
test = test.T
test.shape

(45297, 453)

In [41]:
test.head()
test.shape

(45297, 456)

# ANOVA Analysis

In [25]:
## Perform Anova analysis
stats.f_oneway(train_stage1, train_stage2, train_stage3, train_stage4)

F_onewayResult(statistic=array([7.48610686, 0.40055395, 1.36233809, ..., 0.73926343, 0.25758497,
       1.34441785]), pvalue=array([6.71658549e-05, 7.52670583e-01, 2.53683491e-01, ...,
       5.29040824e-01, 8.55925592e-01, 2.59343623e-01]))

In [ ]:
dfn.head()

In [ ]:
train_stage1.head()

In [28]:
## Get the list of p values out of the dataframe
p_vals = stats.f_oneway(train_stage1, train_stage2, train_stage3, train_stage4).pvalue

#dfn = dfn.reset_index()
test = test.reset_index()

## Get the list of genes that these values correspond to
#gene_list = dfn['gene']
gene_list = test['gene']

## Create list of (gene, p value) pairs
pairs = list(zip(gene_list, p_vals))

## Use liberal p value threshold
threshold_standard_all = list(filter(lambda x: x[1] <= 0.05, pairs))

## Use standard p-value threshold
threshold_liberal_all = list(filter(lambda x: x[1] <= 0.1, pairs))

print(len(threshold_standard_all))
print(len(list(threshold_liberal_all)))

6179
10055


In [29]:
p_vals

array([6.71658549e-05, 7.52670583e-01, 2.53683491e-01, ...,
       5.29040824e-01, 8.55925592e-01, 2.59343623e-01])

In [ ]:
## Plot the genes that are available
y = [i[0] for i in threshold_standard_all]
x = [i[1] for i in threshold_standard_all]

# sns.set(rc={'figure.figsize':(11.7,8.27)})

sns.barplot(x, y)

In [33]:
## Get the list of p values out of the dataframe
p_vals = stats.f_oneway(train_stage1, train_stage2).pvalue

## Get the list of genes that these values correspond to
#gene_list = dfn['gene']
gene_list = test['gene']

## Create list of (gene, p value) pairs
pairs = list(zip(gene_list, p_vals))

## Use liberal p value threshold
threshold_standard12 = list(filter(lambda x: x[1] <= 0.05, pairs))

## Use standard p-value threshold
threshold_liberal12 = list(filter(lambda x: x[1] <= 0.1, pairs))

print(len(threshold_standard12))
print(len(list(threshold_liberal12)))

5322
8661


In [ ]:
## Plot the genes that are available
y = [i[0] for i in threshold_standard12]
x = [i[1] for i in threshold_standard12]

# sns.set(rc={'figure.figsize':(11.7,8.27)})

sns.barplot(x, y)

In [34]:
## Get the list of p values out of the dataframe
p_vals = stats.f_oneway(train_stage1, train_stage3).pvalue

## Get the list of genes that these values correspond to
#gene_list = dfn['gene']
gene_list = test['gene']

## Create list of (gene, p value) pairs
pairs = list(zip(gene_list, p_vals))

## Use liberal p value threshold
threshold_standard13 = list(filter(lambda x: x[1] <= 0.05, pairs))

## Use standard p-value threshold
threshold_liberal13 = list(filter(lambda x: x[1] <= 0.1, pairs))

print(len(threshold_standard13))
print(len(list(threshold_liberal13)))

2655
4495


In [ ]:
## Plot the genes that are available
y = [i[0] for i in threshold_standard13]
x = [i[1] for i in threshold_standard13]

# sns.set(rc={'figure.figsize':(11.7,8.27)})

sns.barplot(x, y)

In [35]:
## Get the list of p values out of the dataframe
p_vals = stats.f_oneway(train_stage1, train_stage4).pvalue

## Get the list of genes that these values correspond to
#gene_list = dfn['gene']
gene_list = test['gene']

## Create list of (gene, p value) pairs
pairs = list(zip(gene_list, p_vals))

## Use liberal p value threshold
threshold_standard14 = list(filter(lambda x: x[1] <= 0.05, pairs))

## Use standard p-value threshold
threshold_liberal14 = list(filter(lambda x: x[1] <= 0.1, pairs))

print(len(threshold_standard14))
print(len(list(threshold_liberal14)))

6241
8794


In [ ]:
## Plot the genes that are available
y = [i[0] for i in threshold_standard14]
x = [i[1] for i in threshold_standard14]

# sns.set(rc={'figure.figsize':(11.7,8.27)})

sns.barplot(x, y)

In [36]:
## Get the list of p values out of the dataframe
p_vals = stats.f_oneway(train_stage2, train_stage3).pvalue

## Get the list of genes that these values correspond to
#gene_list = dfn['gene']
gene_list = test['gene']
## Create list of (gene, p value) pairs
pairs = list(zip(gene_list, p_vals))

## Use liberal p value threshold
threshold_standard23 = list(filter(lambda x: x[1] <= 0.05, pairs))

## Use standard p-value threshold
threshold_liberal23 = list(filter(lambda x: x[1] <= 0.1, pairs))

print(len(threshold_standard23))
print(len(list(threshold_liberal23)))

7513
11022


In [ ]:
## Plot the genes that are available
y = [i[0] for i in threshold_standard23]
x = [i[1] for i in threshold_standard23]

# sns.set(rc={'figure.figsize':(11.7,8.27)})

sns.barplot(x, y)

In [37]:
## Get the list of p values out of the dataframe
p_vals = stats.f_oneway(train_stage2, train_stage4).pvalue

## Get the list of genes that these values correspond to
#gene_list = dfn['gene']
gene_list = test['gene']
## Create list of (gene, p value) pairs
pairs = list(zip(gene_list, p_vals))

## Use liberal p value threshold
threshold_standard24 = list(filter(lambda x: x[1] <= 0.05, pairs))

## Use standard p-value threshold
threshold_liberal24 = list(filter(lambda x: x[1] <= 0.1, pairs))

print(len(threshold_standard24))
print(len(list(threshold_liberal24)))

5575
9426


In [ ]:
## Plot the genes that are available
y = [i[0] for i in threshold_standard24]
x = [i[1] for i in threshold_standard24]

# sns.set(rc={'figure.figsize':(11.7,8.27)})

sns.barplot(x, y)

In [38]:
## Get the list of p values out of the dataframe
p_vals = stats.f_oneway(train_stage3, train_stage4).pvalue

## Get the list of genes that these values correspond to
#gene_list = dfn['gene']
gene_list = test['gene']
## Create list of (gene, p value) pairs
pairs = list(zip(gene_list, p_vals))

## Use liberal p value threshold
threshold_standard34 = list(filter(lambda x: x[1] <= 0.05, pairs))

## Use standard p-value threshold
threshold_liberal34 = list(filter(lambda x: x[1] <= 0.1, pairs))

print(len(threshold_standard14))
print(len(list(threshold_liberal14)))

6241
8794


In [ ]:
## Plot the genes that are available
y = [i[0] for i in threshold_standard34]
x = [i[1] for i in threshold_standard34]

# sns.set(rc={'figure.figsize':(11.7,8.27)})

sns.barplot(x, y)

In [39]:
## Check how many genes are similar between 1/2 and all
print(len(set(threshold_standard12).difference(set(threshold_standard_all))), " out of:", len(threshold_standard12))

## Check how many genes are similar between 1/3 and all
print(len(set(threshold_standard13).difference(set(threshold_standard_all))), " out of:", len(threshold_standard13))

## Check how many genes are similar between 1/4 and all
print(len(set(threshold_standard14).difference(set(threshold_standard_all))), " out of:", len(threshold_standard14))

## Check how many genes are similar between 2/3 and all
print(len(set(threshold_standard23).difference(set(threshold_standard_all))), " out of:", len(threshold_standard23))

## Check how many genes are similar between 2/4 and all
print(len(set(threshold_standard24).difference(set(threshold_standard_all))), " out of:", len(threshold_standard24))

## Check how many genes are similar between 3/4 and all
print(len(set(threshold_standard34).difference(set(threshold_standard_all))), " out of:", len(threshold_standard34))

5307  out of: 5322
2643  out of: 2655
6199  out of: 6241
7496  out of: 7513
5570  out of: 5575
4015  out of: 4036


In [ ]:
# Create and print correlation matrix:
corr = train_stage1.corr()
print(corr)